In [1]:
import os
import re
import json
import glob
import requests
import pandas as pd
from tqdm import tqdm

CONTROLLED_CSV = "controlled_agents_qwen.csv"
DEBATE_CSV     = "debate_results_qwen.csv"
JSONL_GLOB     = "topic_*_qwen.jsonl"
OLLAMA_URL     = "http://localhost:11434/api/generate"
LLAMA_MODEL    = "llama3:8b" 
TIMEOUT_SEC    = 90

controlled = pd.read_csv(CONTROLLED_CSV, encoding="utf-8")

controlled.columns = [c.strip() for c in controlled.columns]

for col in ["region", "occupation"]:
    if col in controlled.columns:
        controlled[col] = controlled[col].astype(str).str.strip()

if "topic_preference" in controlled.columns:
    controlled["topic_preference"] = controlled["topic_preference"].astype(int)

jsonl_files = sorted(glob.glob(JSONL_GLOB))
print(f"✅ Found {len(jsonl_files)} topic files")

debate_df = None
if os.path.exists(DEBATE_CSV):
    debate_df = pd.read_csv(DEBATE_CSV, encoding="utf-8")
    print(f"✅ Loaded debate_results_qwen.csv: {len(debate_df)} rows")


def call_llama_5scale(a_text: str, b_text: str, context_label: str) -> int | None:
    prompt = f"""You are an impartial evaluator.
Rate how much the two people agree with each other on a 1–5 scale,
based ONLY on the following {context_label} texts.

1 = totally disagree, 5 = totally agree.

A: {a_text}

B: {b_text}

Output ONLY a single digit 1-5."""
    try:
        resp = requests.post(
            OLLAMA_URL,
            json={"model": LLAMA_MODEL, "prompt": prompt, "stream": False},
            timeout=TIMEOUT_SEC,
        )
        resp.raise_for_status()
        # Ollama: when stream=False, response JSON = {"response":"...","done":true,...}
        data = resp.json()
        text = data.get("response", "")
        m = re.search(r"\b([1-5])\b", text)
        return int(m.group(1)) if m else None
    except Exception as e:
        print("⚠️ LLaMA error:", e)
        return None


def pick_pref_response(topic_id: int, pref: int, region: str | None, occupation: str | None) -> str | None:
    
    sub = controlled[controlled["topic_id"] == topic_id]
    sub = sub[sub["topic_preference"] == int(pref)]

    if region and "region" in sub.columns:
        region = str(region).strip()
        if (sub["region"] == region).any():
            sub = sub[sub["region"] == region]

    if occupation and "occupation" in sub.columns and len(sub) > 1:
        occupation = str(occupation).strip()
        if (sub["occupation"] == occupation).any():
            sub = sub[sub["occupation"] == occupation]

    if len(sub) == 0:
        return None

    return str(sub.iloc[0]["Preference_Response"])


rows_out = []

for jf in tqdm(jsonl_files, desc="📂 Scoring Qwen topics with LLaMA 3–8B"):
    with open(jf, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            try:
                data = json.loads(line)
            except Exception:
                continue

            topic_id = data.get("topic_id")
            agents   = data.get("agents", [])
            rounds   = data.get("rounds", [])

            if topic_id is None or len(agents) < 2 or len(rounds) == 0:
                continue

            A = agents[0] if isinstance(agents, list) else agents.get("A", {})
            B = agents[1] if isinstance(agents, list) else agents.get("B", {})

            A_pref = A.get("pref")
            B_pref = B.get("pref")
            A_region = str(A.get("region", "")).strip()
            B_region = str(B.get("region", "")).strip()
            A_job    = str(A.get("occupation", "")).strip()
            B_job    = str(B.get("occupation", "")).strip()

            A_pref_resp = pick_pref_response(topic_id, A_pref, A_region, A_job)
            B_pref_resp = pick_pref_response(topic_id, B_pref, B_region, B_job)

            pre_score = None
            if A_pref_resp and B_pref_resp:
                pre_score = call_llama_5scale(A_pref_resp, B_pref_resp, "preference statements")

            last = rounds[-1] if rounds else {}
            A_last = str(last.get("A", ""))
            B_last = str(last.get("B", ""))

            post_score = None
            if A_last and B_last:
                post_score = call_llama_5scale(A_last, B_last, "post-debate final turn")

            delta = None
            if pre_score is not None and post_score is not None:
                delta = post_score - pre_score

            rows_out.append({
                "topic_id": topic_id,
                "A_pref": A_pref, "B_pref": B_pref,
                "A_region": A_region, "B_region": B_region,
                "A_occupation": A_job, "B_occupation": B_job,
                "A_pref_response": A_pref_resp, "B_pref_response": B_pref_resp,
                "pre_agreement": pre_score,
                "post_agreement": post_score,
                "Δagreement": delta,
            })

out_df = pd.DataFrame(rows_out)
cols = [
    "topic_id",
    "A_pref","B_pref",
    "A_region","B_region","A_occupation","B_occupation",
    "A_pref_response","B_pref_response",
    "pre_agreement","post_agreement","Δagreement",
]

cols = [c for c in cols if c in out_df.columns]
out_df = out_df[cols].sort_values(["topic_id"]).reset_index(drop=True)

out_file = "Qwen_dialogue_llama3_score.csv"
out_df.to_csv(out_file, index=False, encoding="utf-8-sig")
print(f"\n✅ Done! Saved {len(out_df)} rows to: {out_file}")
display(out_df.head(8))

✅ Found 6 topic files
✅ Loaded debate_results_qwen.csv: 90 rows


📂 Scoring Qwen topics with LLaMA 3–8B: 100%|█████| 6/6 [06:37<00:00, 66.30s/it]


✅ Done! Saved 90 rows to: Qwen_dialogue_llama3_score.csv


,topic_id,A_pref,B_pref,A_region,B_region,A_occupation,B_occupation,A_pref_response,B_pref_response,pre_agreement,post_agreement,Δagreement
0,1,1,5,Singapore,Germany,Social worker,Social worker,"Reason 1: As a social worker, I prioritize the...","Reason 1: As a social worker, I have direct ex...",2,4,2
1,1,3,3,USA,Mexico,Musician,UX designer,"Reason 1: As a musician, I often work closely ...","Reason 1: As a UX designer, I prioritize user ...",4,4,0
2,1,3,3,Mexico,Singapore,UX designer,Data analyst,"Reason 1: As a UX designer, I prioritize user ...","Reason 1: As a data analyst, I handle sensitiv...",1,4,3
3,1,3,3,India,Singapore,Librarian,Data analyst,"Reason 1: As a librarian, I handle a vast amou...","Reason 1: As a data analyst, I handle sensitiv...",4,4,0
4,1,3,3,Singapore,USA,Social worker,Musician,"Reason 1: As a social worker in Singapore, I u...","Reason 1: As a musician, I often work closely ...",4,4,0
5,1,2,4,USA,Germany,Musician,Social worker,"Reason 1: As a musician, I believe in the impo...","Reason 1: As a social worker in Germany, I und...",3,4,1
6,1,2,4,Germany,Italy,IT manager,Graphic designer,"Reason 1: In my role as an IT manager, I handl...","Reason 1: As a graphic designer, I often handl...",4,4,0
7,1,3,3,Singapore,India,Social worker,Librarian,"Reason 1: As a social worker in Singapore, I u...","Reason 1: As a librarian, I handle a vast amou...",4,3,-1
